# 도커 스웜을 사용하는 이유
- docker ps 명령어는 하나의 도커 엔진에 존재하는 컨테이너의 목록을 출력하며, create, run 명령어 또한 하나의 도커 엔진에 컨테이너를 생성한다.
- 하나의 호스트 머신에서 도커 엔진을 구동하다가 CPU, 메모리, 디스크 용량과 같은 자원이 부족하면 어떻게 해결할까?
- 가장 간단한 해답은 '매우 성능이 좋은 서버를 새로 산다' 이다.
    - 하지만 자원의 확장성 측면이나 비용 측면에서 좋은 해답이 아니다.
    - 자원이 부족할 때마다 더 성능이 좋은 서버를 살 수는 없으며, 높은 가격의 서버를 사고 유지하는 비용 또한 무시할 수 없다.
- 여러 대의 서버를 클러스터로 만들어 자원을 병렬로 확장해야 한다.
    - 스케쥴러, 로드밸런서, 고가용성 등에 대한 문제를 해결해야 한다.
    - 대표적인 오픈소스 솔루션 : 도커 스웜, 스웜 모드

# 도커 스웜 종류

## 스웜 클래식
- 도커 버전 1.6 이후부터 사용할 수 있는 컨테이너로서의 스웜
- 여러대의 도커 서버를 하나의 지점에서 사용하도록 단일 접근점 제공
- docker ps, docker run 등 일반적인 도커 명령어와 도커 API로 클러스터의 서버를 제어하고 관리 할 수 있는 기능 제공
- 분산 코디네이터, 에이전트 등이 별도로 실행되어야 함

## 스웜 모드
- 도커 버전 1.12 이후부터 사용할 수 있는 도커 스웜 모드(Swarm Mode)
- 마이크로 서비스 아키텍처의 컨테이너를 다루기 위한 클러스터링 기능에 초점을 맞춤
- 같은 컨테이너를 동시에 여러개 생성해 필요에 따라 유동적으로 컨테이너의 수를 조절 할 수 있다.
- 서비스 장애에 대비한 고가용성과 부하 분산을 위한 로드밸런싱 기능 제공
- 서비스 확장성과 안정성 등 여러 측면에서 스웜 클래식보다 뛰어남
- 클러스터링을 위한 모든 도구(분산 코디네이터, 에이전트 등)가 도커 엔진 자체에 내장되어 있음 (더욱 쉽게 서버 클러스터 구축 가능)

> 분산 코디네이터
> - 클러스터에 영입할 새로운 서버의 발견, 클러스터의 각종 설정 저장, 데이터 동기화 등에 주로 이용됨
> - etcd, zookeeper, consul 등이 대표적임

# 도커 스웜 모드
- 별도 설치 과정이 필요하지 않으며, 도커 엔진 자체에 내장되어 있음
- docker info 명령어를 통해 도커 엔진의 스웜 모드 클러스터 정보 확인 가능
  ```bash
  $ docker info | grep Swarm
  ```

> 서버 클러스터링을 할 때는 반드시 각 서버의 시각을 NTP 등의 툴을 이용해 동기화 해야 함
> - 서버 간에 설정된 시각이 다를 경우 예상치 못한 오류 발생

## 구조
- 매니저 노드와 워커 노드로 구성되어 있음
- 워커 노드
    - 컨테이너가 생성되고 관리되는 도커 서버
- 매니저 노드
    - 워커 노드를 관리하기 위한 도커 서버
    - 매니저 노드에도 컨테이너가 생성될 수 있음 (기본적으로 워커 노드의 역할 포함)
    - 일반적인 매니저 역할을 하는 노드 + 리더 역할을 하는 노드
    - 리더 매니저는 모든 매니저 노드에 대한 데이터 동기화와 관리 담당 (항상 작동할 수 있는 상태여야 함)
    - 리더 매니저의 서버가 다운되는 등의 장애가 생기면 매니저는 새로운 리더를 선출함(Raft Consensus 알고리즘 사용)
- 매니저 노드는 1개 이상 있어야 하지만, 워커 노드는 없을 수도 있음
    - 매니저 노드가 워커 노드 역할도 포함하고 있어, 매니저 노드만으로 스웜 클러스터를 구성할 수있기 때문
    - 일반적으로 워커 노드와 매니저 노드를 구분해서 사용하는 것을 권장함
- 매니저 노드를 다중화하는 것을 권장함
    - 매니저 노드의 부하를 분산하고 특정 매니저 노드가 다운됐을 때 정상적으로 스웜 클러스터를 유지할 수 있기 때문
- 매니저 노드는 가능한 한 홀수 개로 구성하는 것이 바람직함
    - 매니저 노드의 절반 이상에 장애가 생겨 정상적으로 작동하지 못할 경우, 장애가 생긴 매니저 노드가 복구될 때까지 클러스터 운영을 중단함
    - 홀수로 구성해야 매니저 노드의 장애를 더 허용할 수 있으며, 매니저 노드 사이의 데이터 일관성을 유지할 수 있다.
 
## 클러스터 구축
- `docker swarm init` 명령어를 입력해 매니저 역할을 할 서버에서 스웜 클러스터 시작
    - `--advertise-addr` 에는 다른 도커 서버가 매니저 노드에 접근하기 위한 IP 주소를 입력함
    ```bash
    $ docker swarm init --advertise-addr 192.168.0.100
    ```
- `docker swarm join` 명령어를 입력해 새로운 우커 노드를 스웜 클러스터에 추가 가능
    - `--token` 옵션에 사용된 토큰 값은 새로운 노드를 해당 스웜 클러스터에 추가하기 위한 비밀키이다. (외부 노출하지 말 것)
    - `docker swarm join-token manager` 명령어를 입력해 매니저 노드를 추가하기 위한 토큰 확인
    - `docker swarm join-token worker` 명령어를 입력해 워커 노드를 추가하기 위한 토큰 확인
- `docker node ls` 명령어를 입력해 특정 도커 서버가 정상적으로 스웜 클러스터에 추가됐는지 확인
    - 출력 결과 중 ID 옆에 별표(*)가 붙어있는 노드가 현재 서버이다.
- 실제 운영 환경에서는 주기적으로 스웜 클러스터의 토큰을 변경하는 것이 안전함
    - 토큰 갱신 : `docker swarm join-token --rotate manager`
        - 매니저 노드에서만 수행 가능
- 추가된 워커 노드 삭제
    - `docker swarm leave`
        - 해당 워커 노드의 상태를 Down으로 인지할 뿐 자동으로 워커 노드를 삭제하지는 않는다.
        - 매니저 노드에서 `docker node rm` 명령어 사용해 해당 워커 노드를 삭제해줘야 한다.
- 매니저 노드 삭제
    - `docker swarm leave` 명령어에 `--force` 옵션을 추가해야 삭제된다.
    - 해당 매니저 노드에 저장되어 있던 클러스터의 정보도 삭제된다. (매니저 노드가 단 한개만 존재한다면 스웜 클러스터는 더이상 사용하지 못하게 됨)
- 워커 노드를 매니저 노드로 변경
    - `docker node promote` 명령어 사용
- 매니저 노드를 워커 노드로 변경
    - `docker node demote` 명령어 사용
    - 매니저 노드가 1개일 때는 사용 불가
    - 매니저 리더 노드에 demote 명령어 사용 시, 다른 매니저 노드 중 새로운 리더 선출

## 서비스
- 스웜 모드에서 제어하는 단위
- 같은 이미지에서 생성된 컨테이너의 집합
- 서비스 내에 컨테이너는 1개 이상 존재할 수 있으며, 컨테이너들은 각 워커 노드와 매니저 노드에 할당됨 (이러한 컨테이너들을 태스크라고 부름)
- 예를 들어, ubuntu:14.04 이미지로 서비스를 생성하고 컨테이너 수를 3개로 설정했다고 가정하자.
    - 3개의 노드에 각 1개의 컨테이너가 생성될 수도 있다. (1개 노드에 여러개의 컨테이너가 생성될 수 있음)
    - 함께 생성된 컨테이너를 레플리카(replica)라고 한다.
    - 서비스 내에 정의된 레플리카의 수만큼 컨테이너가 스웜 클러스터에 존재하지 않으면, 새로운 컨테이너 레플리카를 생성한다.
- 롤링 업데이트(Rolling Update) 기능 제공
    - 서비스 내 컨테이너들의 이미지를 일괄적으로 업데이트해야 할 때 컨테이너의 이미지를 순서대로 변경해 서비스 자체가 다운되는 시간 없이 컨테이너의 업데이트 진행 가능

> 롤링 업데이트
> - 여러 개의 서버, 컨테이너 등으로 구성된 클러스터의 설정이나 데이터 등을 변경하기 위해 하나씩 재시작하는 것을 의미함
> - 롤링 업데이트 없이 한번에 모든 서버나 컨테이너를 재시작하면, 제공하는 서비스에 다운 시간(Down Time)이 생긴다.

- 서비스를 제어하는 도커 명령어는 전부 매니저 노드에서만 사용 가능
- 서비스 생성
    - `docker service create`
    ```bash
    $ docker service create --name myweb --replicas 2 -p 80:80 nginx
    ```
    - -p 옵션에 80:80 입력함으로써, 스웜 클러스터 자체에 포트를 개방함
        - 각 노드의 80번 포트로 들어온 요청을 컨테이너 중 1개로 리다이렉트(redirect) 한다. (스웜 모드는 라운드 로빈 방식으로 컨테이너를 결정함)
            - 스웜 클러스터 내의 어떠한 노드로 접근해도 위 서비스의 웹 서버에 접근 가능
                - nginx 컨테이너가 없는 노드로 접근해도 서비스에 접근 가능
- 서비스 내의 컨테이너는 detached 모드로, 즉 docker run 명령어의 -d 옵션을 사용해 동작 가능한 이미지여야 한다.
    - 그렇지 않다면, 컨테이너 내부를 차지하고 있는 프로세스가 없어 컨테이너가 정지되고,
    - 스웜 매니저는 서비스의 컨테이너에 장애가 생긴 것으로 판단해 컨테이너를 계속 반복해서 생성하게 됨
- 서비스 정상 구동 여부 확인
    - `docker service ls`
- 서비스 자세한 정보 확인
    - `dockere service ps [서비스 이름]`
    - 서비스 내의 컨테이너 목록, 상태, 컨테이너가 할당된 노드 위치 등 확인
- 서비스 삭제
    - `docker service rm [서비스 이름]`
    - 서비스 상태에 관계 없이 바로 삭제함
- 서비스 생성을 위해 Private 저장소 또는 레지스트리에서 이미지를 받아올 경우,
    - 매니저 노드에서 로그인 한 뒤, docker service create 명령어에 --with-registry-auth 추가해 사용하면,
    - 워커 노드에서 별도로 로그인하지 않아도 이미지를 받아올 수 있다.
    ```bash
    $ docker service create --with-registry-auth \
    ...
    ```
- 서비스 레플리카 셋의 수 조절
    - `docker service scale`
        - ex. `docker service scale myweb=4`
- Global 서비스
    - 스웜 클러스터 내에서 사용할 수 있는 모든 노드에 반드시 1개의 컨테이너를 생성함
    - 따라서 글로벌 모드로 생성한 서비스는 레플리카셋의 수를 별도로 지정하지 않음
    - 스웜 클러스터를 모니터링하기 위한 에이전트 컨테이너 등을 생성해야 할 때 유용함
    - `docker service cretae --mode global`
        - ex. `docker service create --name global_web --mode global nginx`

> docker ps 도커 클라이언트 명령어로도 스웜의 서비스로 생성된 컨테이너 확인 가능
> - docker ps 명령어에 --filter is-task=true 추가
> - ex. docker ps --filter is-task=true --format {{.Names}}

### 서비스 장애 복구
- 다운됐던 노드를 다시 시작해 정상적인 상태를 회복해도 장애를 복구하기 위해 다른 노드로 옮겨진 컨테이너가 해당 노드에 자동 할당되지 않는다. (Rebalancing 안됨)
    - 새로운 노드를 추가하거나 다운됐던 노드를 다시 복구했을 때 서비스의 컨테이너 할당의 균형을 맞추기 위해서는
    - scale 명령어를 이용해 컨테이너 수를 줄이고 다시 늘려야 함
    ```bash
    $ docker service scale myweb=1
    $ docker service scale myweb=4
    ```

### 서비스 롤링 업데이트
- 스웜 모드는 롤링 업데이트를 자체적으로 지원함
- 테스트
    - 서비스 생성
        - `docker service create --name myweb2 --replicas 3 nginx:1.10`
    - 서비스 이미지 업데이트
        - `docker service update --image nginx:1.11 myweb2`
    - 서비스 내의 컨테이너 목록 확인하면, 각 컨테이너의 이미지가 변경된 것 확인 가능
        - nginx:1.11 이미지와 nginx:1.10 은 이미지 레이어의 차이가 크지 않으므로 빠른 속도로 롤링 업데이트 진행됨

> dockere service ps 명령어에서 NAME 항목에 \_가 붙어있는 컨테이너는 어떤 이유로든 동작을 멈춘 컨테이너로서, 서비스에서의 컨테이너 변경 기록을 나타냄

- 서비스 생성 시, 롤링 업데이트의 주기, 업데이트를 동시에 진행할 컨테이너의 수, 업데이트에 실패했을 때 어떻게 할 것인지 설정 가능
    - 각 컨테이너 레플리카를 10초 단위로 업데이트하며 업데이트 작업을 한번에 2개의 컨테이너에 수행하도록 서비스 생성
        ```bash
        $ docker service create --replicas 4 --name myweb3 --update-delay 10s --update-parallelism 2 nginx:1.10
        ```
    - 롤링 업데이트 설정 정보 확인
        - `docker service inspect --pretty myweb3`